# Limpieza de Transferencias

Este código necesita lo siguiente para que funciones:
- Archivo de transferncias con fecha en el nombre
- Archivo de conversión de unidades en formato csv

In [1]:
import re
import pandas as pd
import numpy as np
import openpyxl as xl
from excel_functions import ExcelFunctions
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side

In [3]:
unidades = pd.read_csv(f'../static_data/conversion_unidades.csv', usecols=['sku', 'pza_cja'])

In [4]:
def set_almacenes_to_cero(x, unwanted_almacenes = [136, 17]):
    '''
    Esta función se enciarga de convertir a 0 todos los almacenes especificados
    '''
    almacen = x.almacen
    if almacen in unwanted_almacenes:
        return 0
    else:
        return x.cantidad

In [5]:
def convierte_a_cajas(x, col_names={'unidad_medida':'um', 'cantidad':'cantidad'}):
    '''
    Esta función convierte todasl as piezas en cajas
    '''
    um = x[col_names['unidad_medida']]
    cantidad = x[col_names['cantidad']]
    if um == 'PZ':
        resp = cantidad / x.pza_cja
    else:
        resp = cantidad
    resp = np.ceil(resp)
    return resp

In [6]:
def clean_planta_name(x, index_pos=-1):
    '''
    Esta función limpia los nombres de las plantas para que no sean tan largos
    '''
    name = x.planta
    try:
        return name.split()[index_pos].lower()
    except:
        print(f'Algo no está bien con el nombre: {name}')
        return None

In [7]:
def extraer_fecha_from_nombre(nombre):
    # Definimos una lista de posibles patrones de fecha que podrían aparecer en el nombre
    patrones_fecha = [
        r'\b\d{1,2}-\d{1,2}-\d{4}\b',   # Formato dd-mm-yyyy
        r'\b\d{1,2}/\d{1,2}/\d{4}\b',   # Formato dd/mm/yyyy
        r'\b\d{4}-\d{1,2}-\d{1,2}\b',   # Formato yyyy-mm-dd
        r'\b\d{4}/\d{1,2}/\d{1,2}\b',   # Formato yyyy/mm/dd
    ]

    # Buscamos los patrones en el nombre y extraemos la primera coincidencia de fecha
    fecha = None
    for patron in patrones_fecha:
        coincidencias = re.findall(patron, nombre)
        if coincidencias:
            fecha = coincidencias[0]
            break

    return fecha

In [10]:
def clean_transferencias(file, dir_location: str, file_name: str, return_data=False):
    '''
    Esta fucnión se encuarga de toda la funcionalidad
    '''
    ef = ExcelFunctions()
    
    df_transferencias = pd.read_excel(file)
    
    columnas_necesarias = ['Nombre venta', 'Articulo', 'Descripción', 'Cantidad', 'UM', 'Almacén']
    fecha = extraer_fecha_from_nombre(file_name)
    
    processed_transfers = (df_transferencias
     [columnas_necesarias]
     .rename(columns={'Nombre venta':'planta', 'Articulo':'sku', 'Cantidad':'cantidad', 'UM': 'um', 'Almacén':'almacen', 'Descripción':'descripcion'})
     .merge(unidades, on='sku', how='left')
     .assign(
         c_=lambda df: df.apply(lambda x: set_almacenes_to_cero(x), axis=1), # Convertimos a 0 los almacenes no deseados,
         cantidad=lambda df: df.apply(lambda x: convierte_a_cajas(x, col_names={'unidad_medida':'um', 'cantidad':'c_'}), axis=1), # Convertimos todo a cajas
         fecha=fecha, # La voy a dejar en formato string
         planta=lambda df: df.apply(clean_planta_name, axis=1)
     ) 
     .drop(['c_', 'pza_cja', 'um'], axis=1)
     .pivot_table(index=['fecha', 'sku', 'descripcion'], columns='planta', values='cantidad', aggfunc=np.sum) # Hacemos el reporte por planta
     .assign(total=lambda x: x.sum(axis=1)) # Sumamos el total
     .reset_index()
     .sort_values('sku')
    )
    
    # Guardamos el archivo generado en formato excel bonito
    saving_name = f"transferencias_plantas_{fecha.replace('-','_')}"
    byte_data, filename = ef.save_and_download_excel_file(
        df=processed_transfers,
        dir_location=dir_location,
        file_name=saving_name,
        sheet_name='Transferncias',
        n_cols_to_bold=3,
        return_data=return_data
    )
    if return_data:
        return byte_data, filename
    
    print(f'{saving_name} updated succesfully')

In [11]:
clean_transferencias(
    '../raw_data/Reporte T2 31-07-2023.xlsx', 
    '../transformed_data',
    'Reporte T2 31-07-2023.xlsx',
    return_data=False
)

transferencias_plantas_31_07_2023 updated succesfully
